In [ ]:
import csv
import requests
import datetime
from pprint import pprint
import time
import pandas as pd
from pprint import pprint as pp
import json

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 300)

https://stackoverflow.com/questions/52129161/create-2-new-columns-with-pandas-and-assign-variables-based-off-the-date

I would like to alter the script below to give me 2 new columns with 3 possible variables. Date | gamePK | Home | Home Rest | Away | Away Rest

The current matches.csv format is Date | gamePK | Home | Away

Home Rest & Away Rest (-1 if the team played the day prior vs a team that didn't, 1 if the team didn't play the day prior vs an opponent who did, 0 otherwise)

Any information on how to create the columns and write this statement for them would be much appreciated.

### make request to NHL stats server for data and save it to a file

In [ ]:
address_p1 = 'https://statsapi.web.nhl.com/api/v1/schedule?site=en_nhl&gamePk=20180'
address_p2 = '&leaderGameTypes=R&expand=schedule.broadcasts.all,schedule.radioBroadcasts,schedule.teams,schedule.ticket,schedule.game.content.media.epg'

with open('data.json', 'w') as outfile:
    
    data_list = []
    
    for i in range(20001,20070):  # end 20070

        req = requests.get(address_p1 + str(i) + address_p2)
        data = req.json()
        
        data_list.append(data)  # append each request to the data list; will be a list of dicts
        
        
    json.dump(data_list, outfile)  # save the json file so you don't have to keep hitting the nhl server with your testing

### read the json file back in

In [ ]:
with open('data.json') as f:
    data = json.load(f)

### this is what 1 record looks like

In [ ]:
for i, x in enumerate(data):
    if i == 0:
        pp(x)

### parse each dict

In [ ]:
kp = []
for json_dict in data:
    for item in json_dict['dates']:
        date = item['date']
        games = item['games']
        for game in games:
            gamePk = game['gamePk']
            season = game['season']
            teams = game['teams']
            home = teams['home']
            home_tm = home['team']['abbreviation']           
            away = teams['away']
            away_tm = away['team']['abbreviation']

            print (date, gamePk, away_tm, home_tm)

            kp.append([date, gamePk, away_tm, home_tm])

### create DataFrame and save to csv

In [ ]:
df = pd.DataFrame(kp, columns=['Date','gamePk','Home', 'Away'])
df.to_csv('matches.csv', sep=',', header=True, index=False)

### read in csv into DataFrame

convert Data to proper datetime format

In [ ]:
df = pd.read_csv('matches.csv', sep=',')
# df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [ ]:
df.head()  # first 5

In [ ]:
df.info()

# On Game Day, What is the Previous Day

In [ ]:
def yesterday(date):
    today = datetime.datetime.strptime(date, '%Y-%m-%d')
    return datetime.datetime.strftime(today - datetime.timedelta(1), '%Y-%m-%d')

In [ ]:
def yesterday_apply(df):
    df['previous_day'] = df.apply(lambda row: yesterday(date=row['Date']), axis=1)

In [ ]:
yesterday_apply(df)

In [ ]:
df

# Did We Play on the Previous Day

In [ ]:
def played_previous_day(df, date, team):
    filter_t = f'(Date == "{date}") & ((Home == "{team}") | (Away == "{team}"))'
    filtered_df = df.loc[df.eval(filter_t)]
    if filtered_df.empty:
        return False  # didn't play previous day
    else:
        return True  # played previous day

In [ ]:
def played_previous_day_apply(df):
    df['home_played_previous_day'] = df.apply(lambda row: played_previous_day(df, date=row['previous_day'], team=row['Home']), axis=1)
    df['away_played_previous_day'] = df.apply(lambda row: played_previous_day(df, date=row['previous_day'], team=row['Away']), axis=1)

In [ ]:
played_previous_day_apply(df)

In [ ]:
df

# Determine Game Day Handicap

Home Rest & Away Rest (-1 if the team played the day prior vs a team that didn't, 1 if the team didn't play the day prior vs an opponent who did, 0 otherwise)

In [ ]:
def handicap(team, home, away):
    if (team == 'home') and not home and away:
        return 1
    elif (team == 'away') and not home and away:
        return -1
    elif (team == 'home') and home and not away:
        return -1
    elif (team == 'away') and home and not away:
        return 1
    else:
        return 0

In [ ]:
def handicap_apply(df):
    df['home_rest'] = df.apply(lambda row: handicap(team='home', home=row['home_played_previous_day'], away=row['away_played_previous_day']), axis=1)
    df['away_rest'] = df.apply(lambda row: handicap(team='away', home=row['home_played_previous_day'], away=row['away_played_previous_day']), axis=1)

In [ ]:
handicap_apply(df)

In [ ]:
df

### data presentation method

In [ ]:
def find_last(match_date, da, team):

        home_play = da[da['Home'] == team].tail(1)  # then find last matches played at home, select greatest
        away_play = da[da['Away'] == team].tail(1)  # "  " find last matches played at away, select greatest

        #then take the last match played, either home or away, whichever is more recent
        if home_play.empty and away_play.empty:
            print (team, "no_matches before this date")
            last_match = 'NA'

        elif home_play.empty:
            last_match = away_play.Date.item()

        elif away_play.empty:
            last_match = home_play.Date.item()            

        else:
            last_match = max([home_play.Date.item(), away_play.Date.item()])


        if last_match != 'NA':

            #And then subtract this from "todays" date (match_date)
            duration_since_last = pd.to_datetime(match_date) - pd.to_datetime(last_match)
            print ("Team:", team)
            print ("Todays game date  = ", match_date)
            print ("Last match played = ", last_match)
            print ("Rest Period       = ", duration_since_last)

            print()

            return duration_since_last

### produce your output

In [ ]:
for k in df.index:

    home_team  = df.Home[k]
    away_team  = df.Away[k]
    match_date = df.Date[k]
    gamePk = df.gamePk[k]

    #we want to find all date values less than todays match date.
    da = df[df['Date'] < match_date]

##    if not da.empty:
    for team in [home_team, away_team]:
        print ("Record", k, home_team, 'vs', away_team)

        find_last(match_date, da, team)  # call your method

    print('_' * 40)